In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


data = pd.read_csv('drive/MyDrive/Colab Notebooks/BankChurners_ML.csv')
data_old = pd.read_csv('drive/MyDrive/Colab Notebooks/BankChurners.csv')

columns = pd.DataFrame(data_old.columns[2:21])
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.0,0.404255,1.0,0.6,0.6,0.5,0.50,0.0,0.604651,0.8,0.166667,0.500000,0.340190,0.308701,0.345116,0.729110,0.035273,0.248062,0.875067,0.061061,0.000086,0.999920
1,1.0,0.382979,1.0,0.4,0.4,0.5,0.25,0.0,0.534884,0.4,0.166667,0.333333,0.077747,0.495431,0.079970,0.751502,0.032158,0.108527,0.455574,0.311311,0.000047,0.999950
2,1.0,0.468085,1.0,0.4,0.4,1.0,0.75,0.0,0.534884,1.0,0.500000,0.500000,0.308900,0.666269,0.289051,0.832332,0.051797,0.170543,0.474960,0.144144,0.000295,0.999710
3,1.0,0.340426,1.0,1.0,1.0,0.5,0.00,0.0,0.418605,0.8,0.500000,0.333333,0.160522,0.582837,0.152928,0.453850,0.038444,0.248062,0.366182,0.217217,0.000183,0.999820
4,1.0,0.829787,1.0,0.2,0.4,0.5,0.25,0.0,0.953488,1.0,0.333333,0.500000,0.231476,0.630513,0.217454,0.782632,0.044731,0.124031,0.734518,0.174174,0.000190,0.999810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057,1.0,0.510638,1.0,0.4,0.4,1.0,0.25,0.0,0.627907,0.4,0.333333,0.500000,0.077536,0.735399,0.062266,0.383943,0.832647,0.829457,0.461497,0.462462,0.000184,0.999820
10058,0.0,0.319149,1.0,0.4,0.4,0.0,0.25,0.0,0.279070,0.6,0.333333,0.500000,0.085819,0.868494,0.060499,0.439104,0.459219,0.457364,0.367798,0.511512,0.995688,0.004311
10059,0.0,0.382979,0.0,0.2,0.6,0.5,1.00,0.0,0.534884,0.8,0.500000,0.666667,0.120042,0.000000,0.156637,0.447297,0.544175,0.387597,0.440495,0.000000,0.998299,0.001699
10060,0.0,0.085106,1.0,0.4,0.4,0.5,0.25,0.0,0.534884,0.6,0.500000,0.500000,0.116172,0.000000,0.152928,0.292190,0.438689,0.403101,0.388799,0.000000,0.997129,0.002875


In [ ]:
x = data[data.columns[1:20]]
y = data[data.columns[0]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(x_train, y_train)

y_pred = random_forest.predict(x_test)
before_feat_sel = f'Before feature selection\naccuracy_score  {metrics.accuracy_score(y_test, y_pred)}\nf1_score        {metrics.f1_score(y_test, y_pred)}\
                  \nprecision_score {metrics.precision_score(y_test, y_pred)}\nrecall_score    {metrics.recall_score(y_test, y_pred)}\n'

# Apply two Feature Selection methods to choose optimal features

First

In [ ]:
best_features = SelectKBest(chi2)
fit = best_features.fit(x, y)
scores = pd.DataFrame(fit.scores_)

features_scores = pd.concat([columns, scores], axis=1)
features_scores.columns = ('Features', 'Score')
features_scores

,Features,Score
0,Customer_Age,0.235852
1,Gender,7.309869
2,Dependent_count,0.523792
3,Education_Level,0.008074
4,Marital_Status,0.299671
5,Income_Category,0.570600
6,Card_Category,0.268528
7,Months_on_book,0.133569
8,Total_Relationship_Count,38.628891
9,Months_Inactive_12_mon,17.105278


Use inbuilt functions of python's classifiers in order to detect most useful and useless features for classification (Embedded models, especially for trees)

In [ ]:
print(f'Most useful: {columns.iat[12, 0]}, {columns.iat[16, 0]}, {columns.iat[18, 0]}\n')
print(f'Most useless: {columns.iat[3, 0]}, {columns.iat[7, 0]}, {columns.iat[13, 0]}')

Most useful: Total_Revolving_Bal, Total_Trans_Ct, Avg_Utilization_Ratio

Most useless: Education_Level, Months_on_book, Avg_Open_To_Buy


In [ ]:
ind = pd.Index([str(i) for i in (3, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20)])
x_best_1 = data[ind]

x_train, x_test, y_train, y_test = train_test_split(x_best_1, y, test_size=0.3, random_state=3)

random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(x_train, y_train)

y_pred = random_forest.predict(x_test)

print(before_feat_sel)

print(f'After feature selection\naccuracy_score  {metrics.accuracy_score(y_test, y_pred)}\nf1_score        {metrics.f1_score(y_test, y_pred)}\
      \nprecision_score {metrics.precision_score(y_test, y_pred)}\nrecall_score    {metrics.recall_score(y_test, y_pred)}')

Before feature selection
accuracy_score  0.9615766810202053
f1_score        0.9772727272727273                  
precision_score 0.9666666666666667
recall_score    0.9881141045958796

After feature selection
accuracy_score  0.9635640940708844
f1_score        0.9783805031446541      
precision_score 0.9707488299531981
recall_score    0.9861331220285261


Second

In [ ]:
estimator = RandomForestClassifier()
selector = RFE(estimator)
selector.fit(x, y)
print(selector.ranking_, '\n')

ind = pd.Index([str(i) for i in (2, 10, 12, 13, 15, 16, 17, 18, 19, 20)])
x_best_2 = data[ind]

x_train, x_test, y_train, y_test = train_test_split(x_best_2, y, test_size=0.3, random_state=3)

random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(x_train, y_train)

y_pred = random_forest.predict(x_test)

print(before_feat_sel)

print(f'After feature selection\naccuracy_score  {metrics.accuracy_score(y_test, y_pred)}\nf1_score        {metrics.f1_score(y_test, y_pred)}\
      \nprecision_score {metrics.precision_score(y_test, y_pred)}\nrecall_score    {metrics.recall_score(y_test, y_pred)}')

[ 1  9  6  7 10  8 11  4  1  5  3  1  1  2  1  1  1  1  1] 

Before feature selection
accuracy_score  0.9615766810202053
f1_score        0.9772727272727273                  
precision_score 0.9666666666666667
recall_score    0.9881141045958796

After feature selection
accuracy_score  0.9629016230539914
f1_score        0.977961432506887      
precision_score 0.9714620797498046
recall_score    0.9845483359746434
